<a href="https://colab.research.google.com/github/victoriaazabel/SEdigitalorganizations/blob/main/VictoriaZabel_Assignment6_pt2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# try two because collab sucks??

import pandas as pd
from google.colab import drive
from google.colab import data_table
import matplotlib.pyplot as plt
import torch
from torch import tensor
data_table.enable_dataframe_formatter()

drive.mount('/content/drive')
data = pd.read_csv("/content/drive/MyDrive/data_homework_6.csv")

data.head(10)

In [ ]:
data.shape

(8124, 23)

In [ ]:
#summary of non numeric columns
data.describe(include=[object])

In [ ]:
#create dummy variables out of categorical variables

data = pd.get_dummies(data)
data.head(10)

,target_F,target_Q,variable_1_C,variable_1_D,variable_1_G,variable_1_L,variable_1_T,variable_1_Y,variable_2_G,variable_2_H,...,variable_21_T,variable_21_W,variable_21_Z,variable_22_E,variable_22_H,variable_22_M,variable_22_N,variable_22_Q,variable_22_V,variable_22_X
0,0,1,0,0,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,1,0
1,1,0,0,0,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
2,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,0,1,0,0,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,1,0
4,1,0,0,0,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
5,1,0,0,0,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
6,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
7,1,0,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,1,0,0,0
8,0,1,0,0,0,0,0,1,0,0,...,0,1,0,0,1,0,0,0,0,0
9,1,0,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,1,0,0,0


In [ ]:
"""
wrong because i only did variable 16 and 19
# try two of added columns
new_cols = ['variable_16_Q',
       'variable_19_F', 'variable_19_G', 'variable_19_M', 'variable_19_O',
       'variable_19_Q']
data[new_cols].head()
"""

In [ ]:
# create dependent variables (target variables), made them tensors with dummies
target_dep = tensor(data.target_Q)


In [ ]:
# create independent variables (predicting variables)
predict_cols = ["target_F", "target_Q"]

predict_in = tensor(data[predict_cols].values, dtype=torch.float)
predict_in.shape

torch.Size([8124, 2])

In [ ]:
# start with linear model, coefficients for all independents
# predictions are calculated by multiplying each row by the coefficients, and finding the sum
torch.manual_seed(442)

n_coeff = predict_in.shape[1]
coeffs = torch.rand(n_coeff)-0.5
coeffs

tensor([-0.4629,  0.1386])

In [ ]:
# actually multiplying the independents
predict_in * coeffs


tensor([[-0.0000,  0.1386],
        [-0.4629,  0.0000],
        [-0.4629,  0.0000],
        ...,
        [-0.4629,  0.0000],
        [-0.0000,  0.1386],
        [-0.4629,  0.0000]])

In [ ]:
# creating predictions by adding up the rows of 21
pred = (predict_in * coeffs).sum(axis = 1)
pred[:10]

tensor([ 0.1386, -0.4629, -0.4629,  0.1386, -0.4629, -0.4629, -0.4629, -0.4629,
         0.1386, -0.4629])

In [ ]:
# loss cuntion for our gradient descent using the dependent variable
loss = torch.abs(pred - target_dep).mean()
loss

tensor(0.6550)

In [ ]:
# creating a function that brings it all together, should've just gone straight to this
def calc_preds(coeffs, indeps):
  return (indeps*coeffs).sum(axis = 1)

def calc_loss(coeffs, indeps, deps):
  return torch.abs(calc_preds(coeffs, indeps) - deps).mean()

In [ ]:
# calcluatet gradients
coeffs.requires_grad_()

loss = calc_loss(coeffs, predict_in, target_dep)
loss

# so taht pytorch calcualtes gradients
loss.backward()

coeffs.grad

tensor([-0.5180, -0.4820])

In [ ]:

#why didn't this work?
# training the linear model: test and train set
from sklearn.model_selection import train_test_split

"""
train_set, test_set = train_test_split(data, test_size = 0.2,  random_state = 42)
train_in,test_in = predict_in[train_set],predict_in[test_set]
train_dep,test_dep = target_dep[train_set],target_dep[test_set]
#len(train_in),len(test_in)
"""

"""
train_set, test_set = train_test_split(data, test_size=0.2, random_state = 42)
len(train_set), len(test_set)
"""

'\ntrain_set, test_set = train_test_split(data, test_size=0.2, random_state = 42)\nlen(train_set), len(test_set)\n'

In [ ]:
# WHY WHY WHY?!?!?!!?!?
#train_in, test_in = predict_in[train_set], predict_in[test_set]
#train_dep, test_dep = target_dep[train_set], target_dep[test_set]

In [ ]:
# try 2
from fastai.data.transforms import RandomSplitter
trn_split,val_split = RandomSplitter(seed = 42)(data)

trn_indep,val_indep = predict_in[trn_split],predict_in[val_split]
trn_dep,val_dep = target_dep[trn_split],target_dep[val_split]
len(trn_indep),len(val_indep)


(6500, 1624)

In [ ]:
# creating four functions: three to be used in the fourth
# once again i probably could've just skipped to this part

"""
dont need this because its changing later
def update_coeffs(coeffs, lr):
    coeffs.sub_(coeffs.grad * lr)
    coeffs.grad.zero_()
"""

def one_epoch(coeffs, lr):
    loss = calc_loss(coeffs, trn_indep, trn_dep)
    loss.backward()
    with torch.no_grad(): update_coeffs(coeffs, lr)
    print(f"{loss:.3f}", end="; ")

#def init_coeffs(): return (torch.rand(n_coeff) - 0.5).requires_grad_()


# fourth
def train_model(epochs=30, lr=0.01):
    torch.manual_seed(442)
    coeffs = init_coeffs()
    for i in range(epochs): one_epoch(coeffs, lr=lr)
    return coeffs

In [ ]:
# measuring accuracy
def acc(coeffs):
  return (val_dep.bool()==(calc_preds(coeffs, val_indep)>0.5)).float().mean()
acc(coeffs)

tensor(0.5277)

In [ ]:
# skipped rest of linear model
# calculating coefficients with matrix multiplication

trn_dep = trn_dep[:,None]
val_dep = val_dep[:,None]

In [ ]:
val_dep

tensor([[[1]],

        [[0]],

        [[0]],

        ...,

        [[0]],

        [[0]],

        [[0]]], dtype=torch.uint8)

In [ ]:
# neural network now
# skipping 64 because i realized that the layers increase

def init_coeffs():
    hiddens = [10, 10]  # <-- set this to the size of each hidden layer you want
    sizes = [n_coeff] + hiddens + [1]
    n = len(sizes)
    layers = [(torch.rand(sizes[i], sizes[i+1])-0.3)/sizes[i+1]*4 for i in range(n-1)]
    consts = [(torch.rand(1)[0]-0.5)*0.1 for i in range(n-1)]
    for l in layers+consts: l.requires_grad_()
    return layers,consts

In [ ]:
import torch.nn.functional as F

def calc_preds(coeffs, indeps):
    layers,consts = coeffs
    n = len(layers)
    res = indeps
    for i,l in enumerate(layers):
        res = res@l + consts[i]
        if i!=n-1: res = F.relu(res)
    return torch.sigmoid(res)

In [ ]:
# update the update coeffs because we separated layres and constants

def update_coeffs(coeffs, lr):
    layers,consts = coeffs
    for layer in layers+consts:
        layer.sub_(layer.grad * lr)
        layer.grad.zero_()

In [ ]:
# training the model
coeffs = train_model(lr = 0.2)

0.504; 0.471; 0.442; 0.412; 0.387; 0.376; 0.430; 0.336; 0.335; 0.346; 0.324; 0.360; 0.266; 0.257; 0.287; 0.334; 0.248; 0.231; 0.233; 0.218; 0.230; 0.261; 0.220; 0.207; 0.199; 0.195; 0.200; 0.229; 0.197; 0.186; 

In [ ]:
# accuracy
acc(coeffs)

tensor(1.)